In [59]:
!pip install soynlp
!pip install konlpy
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
import pandas as pd
import numpy as np
import itertools
from soynlp.noun import LRNounExtractor_v2
from soynlp.tokenizer import LTokenizer
from konlpy.tag import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [61]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/LIFLOW/data/all_df_with_tokens_according_to_topic.csv')

In [62]:
sentences = list(df[df['convrsThema'] == ' 방송/연예 ']['sentence'])

In [63]:
noun_extractor = LRNounExtractor_v2()
nouns = noun_extractor.train_extract(sentences)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 7825 from 125 sents. mem=5.721 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=56308, mem=5.717 Gb
[Noun Extractor] batch prediction was completed for 2227 words
[Noun Extractor] checked compounds. discovered 174 compounds
[Noun Extractor] postprocessing detaching_features : 929 -> 885
[Noun Extractor] postprocessing ignore_features : 885 -> 859
[Noun Extractor] postprocessing ignore_NJ : 859 -> 858
[Noun Extractor] 858 nouns (174 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=5.717 Gb                    
[Noun Extractor] 38.97 % eojeols are covered


In [64]:
noun_score = dict([])
for w, f in nouns.items() :
  noun_score[w] = nouns[w].score

In [65]:
tokenizer = LTokenizer(scores = noun_score)

In [66]:
with open('/content/drive/MyDrive/Colab Notebooks/LIFLOW/data/stop_words_0_100.txt', 'r', encoding='UTF8') as f :
  stop_words = f.readlines()

In [67]:
stop_words = [w[:-1] for w in stop_words]

In [68]:
doc = """
밥 한끼를 제대로 된 밥 한끼를 먹을 수 있다고 생각하면서 감사하게 감사하게 생각하면서  
노후를 같이 보내야 되지 않을까 싶네요 제가 가장 좋아하는 연예인은 별에서 온 그대에 전지현이고 
잔디씨가 좋아하는 연예인은 누구에요  가장 좋아하는 프로그램은 저 같은 경우는 아는 형님이나 해피투게더 같은 
(SP:에능)예능 프로그램을 좋아하는데  어떤 프로그램을 좋아하나요 유튜브를 좋아하나요 나도 
가끔가다 유튜브를 통해 뮤직비디오를 보긴 해요 어떤 장르를 좋아해요 뮤직비디오 같은 경우요 유튜브에서 볼 때는 
그럼 혹시 티비 프로그램은요 예능 프로그램 중에서 좋아하는 프로는 혹시 있나요 넷플릭스를 좋아하는군요 저도 넷플릭스를 
가끔가다 보긴 해요 요즘 가장 최근에 봤던 넷플릭스 드라마나 아니면 영화 중에서 기억나는 게 혹시 있을까요 그러면은 넷플릭스를 
주로 보고 유튜브를 많이 보는군요 나같은 경우는 최근에 봤던 영화 중에서 기억나는 게 있어요 (SP:에스맨)엑스맨 새로 나온 시리즈를 보고서 
참 감명 깊고 재밌게 봤어요 혹시 본 영화 중에서 뭐 기억 남는 거 뭐 혹시 있나요  좋아하는 프로나 혹시 음악 장르 같은 게 혹시 (SP:있을) 
있을까요 아 그냥 내용에 따라 다른가요 내용이 좋아야죠 이제 사람에 따라 이제 음악도 약간 달라지네요  나 같은 경우는 발라드를 좋아하고 
가끔가다 연예인도 좋아해요 이제 걸 그룹들도 가끔가다 보긴 해요 그런 점도 우리 둘은 약간 차이가 있네요  그러면은 바다씨가 생각하는 
예능 프로그램의 발전 방향이 있나요 이렇게 되면은 예능이 조금 더 발전될 수 있을까  그런 뭐 따로 생각하는 게 혹시 있나요  좋아하는 
동물들 중에서 개나 고양이 같은 게 있잖아요 그럼 요즘 방송 같은 거를 보니까 고양이를 부탁해  그런 예능 프로그램도 이제 간혹가다가 
반려동물이 나오잖아요 그럼 이제 주인하고 그 끈끈한 그런 걸 보여주는데  그런 프로 중에서 보시는 게 있을까요  근데 프로그램 중에서 
동물농장이라든가  주말마다 하는 예능 방송 같은 경우 챙겨 보는 사람들도 꽤 많던데 약간의 차이가 있네요 넷플릭스는 잘 안봐요  
개인적으로 볼 시간도 없고 가끔가다 공중파 방송 뉴스나  뭐 해피투게더 아는 형님 같은 거 이제 간단하게 즐기는 방송을 보지 
이름은 몇번 들어는 봤는데 넷플릭스는 잘 몰라요 스릴러는 본 적은 없어요 아까도 얘기했지만은 액션이나 판타지 히어로물 그런거 
위주로 보고  연애 프로그램은 이나 애니메이션 그런 거 전혀 안봐요 주로 히어로물 아니면 액션물을 많이 봐요 그런 영화가 많죠 근데 요즘엔 
마블 (SP:명화사가)영화사가 마블에서 나오는 건 거의 흥행에 성공하고 광고효과도 뒷받침이 되기 때문에 스파이더맨이라든가 어벤져스라든가 
캡틴 마블이라든가 유명한 작품들은 거의 극장 내에서 인기가 굉장히 많죠 순위 꼭대기에 올라가 있는 인기 프로니까 그런 거 위주로 보죠 
많이 보게 되죠 그건 왜냐하면은 이제 사회에서 받는 스트레스나 이제 일반적인 사회생활하다 보면 시간이 없잖아요 그러다 보니까 일일이 
찾아볼 필요 없이 극장에서 보죠 제일 (SP:하다다는)핫하다는 흥행 순위를 많이 보게 되는데 흥행순위가  마블 영화사 위주로 거의 많이 
형성이 되어 있잖아요 아무래도 좀 없지 않아 그런 현상이 있겠죠 그렇죠 얼마 전에 방금 얘기했던 스파이더맨 홈커밍 (SP:)프로그램  
영화를 봤는데 거기서는 마찬가지로 어벤져스에 바로 이어 지던데 없지 않아 그런 면도 있어요  근데 문제는 뭐냐면은 젊은 사람들 위주로
흥행 영화 흥행이 형성이 되는 거고  그러다 보니까 마블이라는 영화는 곧 흥행이다  근데 마블 캐릭터가 자체가 워낙 또 지배적으로 인기가 많이 있고 
뭐 어벤젼스라든가 아이언맨 스파이더맨 다음에  닥터 스트레인지 그런 프로그램 다 봤으니까요 기타 다른 영화들도 그런 영향이 많이 있겠죠 
그러면 나래씨가 좋아하는 드라마는 어떤 것들이 있어요 먹방같은 경우는 가끔가다 보는데 먹방도 봐요 백종원같은 경우는 백종원은 그 자체가 
브랜드잖아요 그래서 아무래도 피피엘 같은 경우가 많이 들어가 있더라고요 맞아요 정말 그래요
"""

In [69]:
tokenized_nouns1 = tokenizer.tokenize(doc)
tokenized_nouns1 = [w if w not in stop_words else '' for w in tokenized_nouns1]
tokenized_nouns1 = ' '.join(tokenized_nouns1)
print(tokenized_nouns1)

tokenizer2 = Okt()
tokenized_nouns2 = tokenizer2.nouns(doc)
tokenized_nouns2 = [w if w not in stop_words else '' for w in tokenized_nouns2]
tokenized_nouns2 = ' '.join(tokenized_nouns2)
print(tokenized_nouns2)



밥 한끼를 제대로 된 밥 한끼를 먹을  있다고  하면서 감사  감사   하면서 노후를  보내야 되지 않을까 싶네요 제가 가장   연예인  별에서 온 그대  전지현이고 잔디 씨가   연예인  누구에요 가장   프로그램  저  경우  아는 형님 이나 해피투게더  (SP:에능)예능 프로그램   하는데 어떤 프로그램   하나요 유튜브   하나요  가끔 가다 유튜브  통해 뮤직비디오  보긴 해요 어떤 장르   해요 뮤직비디오  경우 요 유튜브  볼 때는 그럼 혹시 티비 프로그램 은요 예능 프로그램 중에서   프로  혹시 있나요 넷플릭스   하는군요 저도 넷플릭스  가끔 가다 보긴 해요  가장 최근에 봤던 넷플릭스 드라마  아니면 영화 중에서 기억   혹시 있을까요 그러면은 넷플릭스  주로  유튜브   보는군요 나같은 경우  최근에 봤던 영화 중에서 기억   있어요 (SP:에스맨)엑스맨 새로 나온 시리즈를 보고서  감명 깊고 재밌게 봤어요 혹시 본 영화 중에서  기억 남는   혹시 있나요   프로  혹시 음악 장르   혹시 (SP:있을) 있을까요 아  내용  따라 다른가요 내용   야죠    따라  음악  약간 달라지네요   경우  발라드    가끔 가다 연예인   해요   그룹들도 가끔 가다 보긴 해요  점도  둘은 약간 차이  있네요 그러면은 바다 씨가   예능 프로그램  발전 방향이 있나요  되면은 예능  조금  발전 될  있을까   따로    혹시 있나요   동물들 중에서 개나 고양이   있잖아요 그럼  방송  거를  고양이  부탁해  예능 프로그램   간혹가다가 반려동물이 나오잖아요 그럼  주인하고  끈끈한   보여주는데  프로 중에서 보시는  있을까요  프로그램 중에서 동물농장 이라든가 주말 마다  예능 방송  경우 챙겨 보는   꽤 많던데 약간  차이  있네요 넷플릭스   안봐요 개인적으로 볼 시간  없고 가끔 가다 공중파 방송 뉴스   해피투게더 아는 형님    간단하게 즐기는 방송  보지 이름  몇번 들어는 봤는데 넷플릭스   몰라요 스릴러는 본 적은 없

In [77]:
#n_gram_range = (1,)
n_gram_range = (2,3)
#count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
count1 = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns1])
candidates1 = count1.get_feature_names_out()

print('trigram 개수 :',len(candidates1))
print('trigram 다섯개만 출력 :',candidates1[:5])

count2 = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns2])
candidates2 = count2.get_feature_names_out()

print('trigram 개수 :',len(candidates2))
print('trigram 다섯개만 출력 :',candidates2[:5])

trigram 개수 : 791
trigram 다섯개만 출력 : ['sp 명화사가' 'sp 명화사가 영화사가' 'sp 에능' 'sp 에능 예능' 'sp 에스맨']
trigram 개수 : 431
trigram 다섯개만 출력 : ['가다가 반려동물' '가다가 반려동물 주인' '가요 내용' '가요 내용 음악' '가장 연예인']


In [78]:
model1 = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding1 = model1.encode([doc])
candidate_embeddings1 = model1.encode(candidates1)

model2 = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding2 = model2.encode([doc])
candidate_embeddings2 = model2.encode(candidates2)

In [80]:
top_n = 8
distances1 = cosine_similarity(doc_embedding1, candidate_embeddings1)
keywords1 = [candidates1[index] for index in distances1.argsort()[0][-top_n:]]
print(keywords1)

top_n = 8
distances2 = cosine_similarity(doc_embedding2, candidate_embeddings2)
keywords2 = [candidates2[index] for index in distances2.argsort()[0][-top_n:]]
print(keywords2)

['극장 내에서 인기가', '간혹가다가 반려동물이 나오잖아요', '인기 프로 니까', '가끔 가다 연예인', '연예인 별에서 그대', '뮤직비디오 보긴 해요', '주로 유튜브 보는군요', '즐기는 방송']
['넷플릭스 주로 유튜브', '인기 프로', '인기 벤젼스', '넷플릭스 스릴러', '유튜브 통해 뮤직비디오', '예능 프로그램 간혹', '해피투게더 예능 프로그램', '인기 프로 위주']
